In [69]:
import json
import pandas as pd
import boto3
from boto3.dynamodb.conditions import Key
from time import gmtime, strftime
from creds import *
import random
from datetime import date

# Connect to AWS DynamoDB

In [2]:
dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name = 'us-west-2'
    )

client = boto3.client(
    'dynamodb',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name = 'us-west-2'
    )

# Create the DynamoDB table

In [31]:
table = dynamodb.create_table(
    TableName='clin_test',
    KeySchema=[
        {
            'AttributeName': 'domain',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'studyid_subjid',
            'KeyType': 'RANGE'
        },
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'domain',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'studyid_subjid',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

table.wait_until_exists()
print(table.item_count)

0


# Insert data into table

In [64]:
table = dynamodb.Table('clin_test')
VISITS = ["VISIT 1","VISIT 2","VISIT 3"]
TRT01A = ["Pembro","Placebo"]
PARAMLBL = ["Alkaline Phosphatase (IU/L)","Aspartate Aminotransferase (IU/L)","Alanine Aminotransferase (IU/L)"]
ABLFL = ["Y","N"]
with table.batch_writer() as batch:
    for i in range(50,100):
        aval = random.randrange(0,100)
        batch.put_item(
            Item={
                'domain': 'adlb',
                'studyid_subjid': 'mid234567'+'_'+str(i),
                'USUBJID': str(i),
                'ABLFL': random.choices(ABLFL,k=1)[0],
                'AVISIT': random.choices(VISITS,k=1)[0],
                'TRT01A': random.choices(TRT01A,k=1)[0],
                'PARAMLBL': random.choices(PARAMLBL,k=1)[0],
                'AVAL': aval
            }
        )



In [96]:
table = dynamodb.Table('clin_test')
VISITS = ["VISIT 1","VISIT 2","VISIT 3"]
AETERM = ["Headache","Fever","Hematoma"]
AESER = ["Y","N"]
with table.batch_writer() as batch:
    for i in range(550,600):
        start_dt = date.today().replace(day=1, month=1).toordinal()
        end_dt = date.today().toordinal()
        AESTDT = date.fromordinal(random.randint(start_dt, end_dt))
        batch.put_item(
            Item={
                'domain': 'adae',
                'studyid_subjid': 'mid234567'+'_'+str(i),
                'USUBJID': str(i),
                'AESER': random.choices(AESER,k=1)[0],
                'AETERM': random.choices(AETERM,k=1)[0],
                'AESTDT': str(AESTDT)
            }
        )


# All the data in schema-less format 

In [66]:
table = dynamodb.Table('clin_test')

response = table.scan()
data = response['Items']
pd.json_normalize(response['Items'],max_level=0)



,AAGEU,AAGE,SEX,USUBJID,studyid_subjid,domain,AGE,AVISIT,PARAMLBL,ABLFL,TRT01A,AVAL
0,Years,77,F,0,mid123456_0,adsl,77,NaN,NaN,NaN,NaN,NaN
1,Years,89,M,1,mid123456_1,adsl,89,NaN,NaN,NaN,NaN,NaN
2,Years,95,M,10,mid123456_10,adsl,95,NaN,NaN,NaN,NaN,NaN
3,Years,66,M,11,mid123456_11,adsl,66,NaN,NaN,NaN,NaN,NaN
4,Years,82,F,12,mid123456_12,adsl,82,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
145,NaN,NaN,NaN,95,mid234567_95,adlb,NaN,VISIT 2,Alkaline Phosphatase (IU/L),N,Pembro,43
146,NaN,NaN,NaN,96,mid234567_96,adlb,NaN,VISIT 2,Alanine Aminotransferase (IU/L),N,Pembro,65
147,NaN,NaN,NaN,97,mid234567_97,adlb,NaN,VISIT 1,Alanine Aminotransferase (IU/L),Y,Placebo,74
148,NaN,NaN,NaN,98,mid234567_98,adlb,NaN,VISIT 2,Aspartate Aminotransferase (IU/L),N,Pembro,5


# Wrap dynamo querying func.

In [62]:
def query_dynamo(domain,studyid):
    if studyid == "all":
        response = table.query(
        KeyConditionExpression=Key('domain').eq(str(domain))
        )

        item = response['Items']
        df = pd.DataFrame.from_dict(item)
        dat = df.drop(['studyid_subjid', 'domain'], axis=1)
    else:
        response = table.query(
        KeyConditionExpression=Key('domain').eq(str(domain)) & Key('studyid_subjid').begins_with(str(studyid))
        )

        item = response['Items']
        df = pd.DataFrame.from_dict(item)
        dat = df.drop(['studyid_subjid', 'domain'], axis=1)
    return dat

# Query one domains data for all studies

In [108]:
query_dynamo("adsl","all")

,AAGEU,AAGE,SEX,USUBJID,AGE
0,Years,77,F,0,77
1,Years,89,M,1,89
2,Years,95,M,10,95
3,Years,66,M,11,66
4,Years,82,F,12,82
...,...,...,...,...,...
95,Years,69,F,5,69
96,Years,62,M,6,62
97,Years,92,F,7,92
98,Years,72,M,8,72


# Query one study and one domains data

In [105]:
query_dynamo("adsl","mid123456").head()

,AAGEU,AAGE,SEX,USUBJID,AGE
0,Years,77,F,0,77
1,Years,89,M,1,89
2,Years,95,M,10,95
3,Years,66,M,11,66
4,Years,82,F,12,82


In [106]:
query_dynamo("adlb","mid234567").head()

,AVISIT,PARAMLBL,ABLFL,USUBJID,TRT01A,AVAL
0,VISIT 3,Aspartate Aminotransferase (IU/L),N,50,Placebo,2
1,VISIT 1,Aspartate Aminotransferase (IU/L),Y,51,Pembro,58
2,VISIT 3,Alkaline Phosphatase (IU/L),Y,52,Placebo,75
3,VISIT 3,Aspartate Aminotransferase (IU/L),N,53,Pembro,80
4,VISIT 3,Aspartate Aminotransferase (IU/L),Y,54,Placebo,52


In [107]:
query_dynamo("adae","mid234567").head()

,AESER,AESTDT,AETERM,USUBJID
0,N,2022-06-30,Hematoma,550
1,Y,2022-03-27,Fever,551
2,Y,2022-04-18,Hematoma,552
3,N,2022-07-14,Headache,553
4,N,2022-09-05,Headache,554
